In [2]:
import numpy as np

import pandas as pd
import statistics
import math

In [265]:
def func_precision(stringList, answer):
    goal_count = 0
    found = 0
    for result in stringList:
        if result and int(result) == int(answer):
            found = 1
        goal_count += 1

    return found/(goal_count-1)

def func_recall(stringList, answer):
    found = 0
    for result in stringList:
        if result and int(result) == int(answer):
            found = 1
            break
    return found

# def func_f1(total, stringList, answer):
#     tp = 0
#     tn = 0
#     fp = 0
#     fn = 0
#     for result in stringList[0:-1]:
#         if result and int(result) == int(answer):
#             tp += 1
#         else:
#             fp += 1
    
#     fn = 1 - tp
    
#     # total is the number of all goals
#     tn = total - tp - fp - fn
#     return 2*tp/(2*tp + fp + fn)

def func_accuracy(total, stringList, answer):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for result in stringList[0:-1]:
        if result == str(answer):
            tp += 1
        else:
            fp += 1
    
    fn = 1 - tp
    
    # total is the number of all goals
    tn = total - tp - fp - fn
    return (tp + tn)/(tn + tp + fp + fn)

def func_accuracy(total, stringList, answer):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for result in stringList[0:-1]:
        if result and int(result) == int(answer):
            tp += 1
        else:
            fp += 1
    
    fn = 1 - tp
    
    # total is the number of all goals
    tn = total - tp - fp - fn
    return (tp + tn)/(tn + tp + fp + fn)


def func_bacc(total, stringList, answer):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for result in stringList[0:-1]:
        if result and int(result) == int(answer):
            tp += 1
        else:
            fp += 1
    
    fn = 1 - tp
    
    # total is the number of all goals
    tn = total - tp - fp - fn

    tpr = tp/(tp + fn)
    tnr = tn/(tn + fp)
    bacc = (tpr + tnr)/2

    return bacc




# return a list of each statistics for every testing case
def calculate_statistics(rows):
    length = rows.shape[0]

    precision = []
    recall = []
    f1_score = []
    accuracy = []
    b_accuracy = []
        
    for index, row in rows.iterrows():
        
        answer = row["Real_Goal"]
        results = row["Results"].split("/")
        all_candidates = row["Cost"].split("/")
        
        total = len(all_candidates)-1   # the last one is /
        
        p = func_precision(results, answer)
        r = func_recall(results, answer)
        f = func_f1(total, results, answer)
        a = func_accuracy(total, results, answer)
        bacc = func_bacc(total, results, answer)
        
        precision.append(p)
        recall.append(r)
        f1_score.append(f)
        accuracy.append(a)
        b_accuracy.append(bacc)
    
    return precision, recall, f1_score, accuracy, b_accuracy


def calculate_statistics_ensemble(rows1, rows2):
    length = rows1.shape[0]

    precision = []
    recall = []
    f1_score = []
    accuracy = []
    b_accuracy = []
        
    for index, _ in rows1.iterrows():
        answer = rows1.at[index, "Real_Goal"]
        results1 = rows1.at[index, "Results"].split("/")
        results2 = rows2.at[index, "Results"].split("/")
        intersection = list(set(results1).intersection(results2))
        if len(intersection) == 1:   ## only have "", empty
            intersection = results1  ## let rows1 always be PM
        all_candidates = rows1.at[index, "Cost"].split("/")
        results = intersection
        results.sort(reverse=True)
        total = len(all_candidates)-1   # the last one is /
        
        p = func_precision(results, answer)
        r = func_recall(results, answer)
        f = func_f1(total, results, answer)
        a = func_accuracy(total, results, answer)
        bacc = func_bacc(total, results, answer)
        precision.append(p)
        recall.append(r)
        f1_score.append(f)
        accuracy.append(a)
        b_accuracy.append(bacc)
    
    return precision, recall, f1_score, accuracy, b_accuracy

sid = 1
df1 = pd.read_csv("pm/f1_def_results_%s.csv" % sid)
df2 = pd.read_csv("lda/dynamic_lda_sub_%s.csv" % sid)
p,r,f1,a,bacc = calculate_statistics_ensemble(df1, df2)

In [319]:
sub_ids = [1,2,3,4,5,6,7,8,9,10]

# pm/f1_def_results_%s.csv
# lda/static_lda_sub_%s.csv
# lstm/lstm_sub_%s.csv

total_collect = []
for sid in sub_ids:
    #df1 = pd.read_csv("pm/f1_def_results_%s.csv" % sid)
    #df2 = pd.read_csv("lda/dynamic_lda_sub_%s.csv" % sid)
    #p,r,f1,a,bacc = calculate_statistics_ensemble(df1, df2)
    dfr = pd.read_csv("lda/static_lda_sub_%s.csv" % sid)
    p,r,f1,a,bacc = calculate_statistics(dfr)
    metric_list = r
    p10 = []
    p30 = []
    p50 = []
    p70 = []
    p100 = []
    for i in range(len(metric_list)):
        if i%5 == 0:
            p10.append(metric_list[i])
        if i%5 == 1:
            p30.append(metric_list[i])
        if i%5 == 2:
            p50.append(metric_list[i])
        if i%5 == 3:
            p70.append(metric_list[i])
        if i%5 == 4:
            p100.append(metric_list[i])
        
    print(statistics.mean(p10))
    total_collect.append(statistics.mean(p10))
    print(statistics.mean(p30))
    total_collect.append(statistics.mean(p30))
    print(statistics.mean(p50))
    total_collect.append(statistics.mean(p50))
    print(statistics.mean(p70))
    total_collect.append(statistics.mean(p70))
    #print(statistics.mean(p100))
    #total_collect.append(statistics.mean(p100))
    
print("---")
print(statistics.mean(total_collect))

_, _, margin_of_error= confidence_interval(total_collect)
print(margin_of_error)

0.4111111111111111
0.37777777777777777
0.45555555555555555
0.6111111111111112
0.28888888888888886
0.3333333333333333
0.24444444444444444
0.3111111111111111
0.32222222222222224
0.34444444444444444
0.3333333333333333
0.3333333333333333
0.35555555555555557
0.43333333333333335
0.4444444444444444
0.5555555555555556
0.24444444444444444
0.2777777777777778
0.2111111111111111
0.4222222222222222
0.36666666666666664
0.3888888888888889
0.34444444444444444
0.6111111111111112
0.35555555555555557
0.32222222222222224
0.4666666666666667
0.5444444444444444
0.37777777777777777
0.4
0.4111111111111111
0.7
0.37777777777777777
0.34444444444444444
0.34444444444444444
0.5555555555555556
0.25555555555555554
0.23333333333333334
0.2777777777777778
0.5888888888888889
---
0.3894444444444444
0.036867251271790545


In [330]:
# overall of each subject
for subject_id in [1,2,3,4,5,6,7,8,9,10]:
    data = pd.read_csv("pm/f1_def_results_%s.csv"%subject_id)
    p, r, f1, a, bacc = calculate_statistics(data)
    
    metric = r
    remove_100 = []
    for i in range(len(metric)):
        if i%5 != 4:
            remove_100.append(metric[i])
    lower, upper, margin_of_error= confidence_interval(remove_100)
    mean = statistics.mean(remove_100)
    print("%s$\pm$%s" % (round(mean, 3), round(margin_of_error,3)))
    # print(round(mean, 3))

0.814$\pm$0.04
0.95$\pm$0.023
0.844$\pm$0.038
0.889$\pm$0.033
0.856$\pm$0.036
0.725$\pm$0.046
0.772$\pm$0.044
0.731$\pm$0.046
0.781$\pm$0.043
0.894$\pm$0.032


In [308]:
## each level of observation averaged over all subjects
sub_ids = [1,2,3,4,5,6,7,8,9,10]
p10 = []
p30 = []
p50 = []
p70 = []
p100 = []
# pm/f1_def_results_%s.csv
# lda/static_lda_sub_%s.csv
# lstm/lstm_sub_%s.csv
for sid in sub_ids:
    dfr = pd.read_csv("lda/static_lda_sub_%s.csv" % sid)
    p,r,f1,a,bacc = calculate_statistics(dfr)
    metric_list = p

    for i in range(len(metric_list)):
        if i%5 == 0:
            p10.append(metric_list[i])
        if i%5 == 1:
            p30.append(metric_list[i])
        if i%5 == 2:
            p50.append(metric_list[i])
        if i%5 == 3:
            p70.append(metric_list[i])
        if i%5 == 4:
            p100.append(metric_list[i])

for metric in [p10,p30,p50,p70,p100]:    
    lower, upper, margin_of_error= confidence_interval(metric)
    mean = statistics.mean(metric)
    print("%s$\pm$%s" % (round(mean, 3), round(margin_of_error,3)))

# print(statistics.mean(p10))
# print(statistics.mean(p30))
# print(statistics.mean(p50))
# print(statistics.mean(p70))
# print(statistics.mean(p100))

0.336$\pm$0.031
0.346$\pm$0.031
0.353$\pm$0.031
0.523$\pm$0.033
0.709$\pm$0.03


In [5]:
# dft = pd.read_csv("PMGR/sub1/kmeans60_f32_results_1.0.csv")
# dft = pd.read_csv("PMGR_new/sub8/kmeans100_f34_sub8.csv")
dft = pd.read_csv("lstm/lstm_sub_1.csv")
p,r,f1,a,bacc = calculate_statistics(dft)

# hypothesis tests

In [325]:
def aver90cases(file):
    total_collect = []
    for sid in sub_ids:
        dfr = pd.read_csv(file)
        p,r,f1,a,bacc = calculate_statistics(dfr)
        metric_list = r   ####
        p10 = []
        p30 = []
        p50 = []
        p70 = []
        p100 = []
        for i in range(len(metric_list)):
            if i%5 == 0:
                p10.append(metric_list[i])
            if i%5 == 1:
                p30.append(metric_list[i])
            if i%5 == 2:
                p50.append(metric_list[i])
            if i%5 == 3:
                p70.append(metric_list[i])
            if i%5 == 4:
                p100.append(metric_list[i])

        total_collect.append(statistics.mean(p10))
        total_collect.append(statistics.mean(p30))
        total_collect.append(statistics.mean(p50))
        total_collect.append(statistics.mean(p70))
        # total_collect.append(statistics.mean(p100))
    return total_collect


from scipy import stats
sub_ids = [1,2,3,4,5,6,7,8,9,10]
for subject_id in sub_ids:
    file1 = "pm/f1_def_results_%s.csv"%subject_id
    #file2 = "lstm/lstm_sub_%s.csv"%subject_id
    file2 = "lda/static_lda_sub_%s.csv"%subject_id
    cluster1_data = aver90cases(file1)
    cluster2_data = aver90cases(file2)

# Perform t-test
t_statistic, p_value = stats.ttest_ind(cluster1_data, cluster2_data)

# Check significance
alpha = 0.05  # Significance level
if p_value < alpha:
    print("The clusters are significantly different.")
else:
    print("The clusters are not significantly different.")

print(p_value)
print(format(p_value, "e"))
print(format(round(p_value, 3), ".3e"))

The clusters are significantly different.
1.18773383024453e-34
1.187734e-34
0.000e+00


# Select N features and M clusters

In [183]:
import os
import sys

# "manual", "kmeans"
classifier_option = "kmeans"
selected_num_features = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
    21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47]
# diff
clusters_num = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]

selected_nf = []
selected_nc = []
max_metric_list = []

for sid in [1,2,3,4,5,6,7,8,9,10]:
    results_dir = "PMGR_new/sub%s"%sid
    cwd = os.getcwd()
    
    max_nf = 0
    max_nc = 0
    max_metric = 0
    
    for fn in selected_num_features:
        for cl in clusters_num:

            csv_file = "%s%s_f%s_sub%s.csv" % (classifier_option, str(cl), str(fn), str(sid) )
            abs_path_csv_file = os.path.join(cwd, results_dir, csv_file)
            if os.path.exists(abs_path_csv_file):
                data = pd.read_csv(abs_path_csv_file)
                p, r, f1, a, bacc = calculate_statistics(data)
                metric = statistics.mean(f1)  #####
                if metric > max_metric:
                    max_nf = fn
                    max_nc = cl
                    max_metric = metric
                
            else:
                print(csv_file + "    Not Exist")
                
    selected_nf.append(max_nf)
    selected_nc.append(max_nc)
    max_metric_list.append(max_metric)


print(selected_nf)  ### features
print(selected_nc)  ### clusters
print(max_metric_list)

[29, 1, 2, 34, 32, 28, 22, 34, 23, 28]
[70, 10, 150, 50, 90, 160, 80, 100, 100, 170]
[0.5766666666666667, 0.5348148148148149, 0.6392592592592593, 0.5214814814814814, 0.5840740740740741, 0.5622222222222222, 0.5307407407407407, 0.5844444444444444, 0.5574074074074074, 0.6037037037037037]


In [184]:
statistics.mean(max_metric_list)

0.5694814814814815

# Alignments calculation times per goal

In [54]:
def timePerGoal(rows):
    length = rows.shape[0]
    g_time = [] 
    for index, row in rows.iterrows():
        stime = row["STime"].split("/")
        time_string = stime[0:-1]  # the last one is /
        for t in time_string:
            g_time.append(int(t))
    
    return g_time


df_pm_t = pd.read_csv("pm/f1_def_results_1.csv")
tg_list = timePerGoal(df_pm_t)
statistics.mean(tg_list)

52197745.72592592

# Show PRIM parameters

In [257]:
from ema_workbench import load_results


delta_list  = []
lamb_list  = []
phi_list = []
theta_list  = []

f1_list = []

sub_id = 1
for sub_id in [1,2,3,4,5,6,7,8,9,10]:
    gz = "prim/f1_100_scenarios_sub%s_.tar.gz" % sub_id
    experiments, outcomes = load_results(gz)

    value = outcomes["f1"]
    max_metric_id = np.argmax(value, axis=0)
    f1 = np.max(value, axis=0)
    # because in experiments there is a line of column name
    e = experiments.iloc[max_metric_id]

    delta_list.append(e["delta"])
    lamb_list.append(e["lamb"])
    phi_list.append(e["phi"])
    theta_list.append(e["theta"])
    f1_list.append(f1)

In [249]:
metric = "f1"

gz = "prim/f1_100_scenarios_sub9_.tar.gz"
experiments, outcomes = load_results(gz)
experiments[metric] = outcomes[metric]
num = int(len(experiments.index) * 0.01)
df = experiments.sort_values(by=[metric], ascending=False).head(num)

df.tail(1)

,delta,lamb,phi,theta,scenario,policy,model,f1
31,0.093954,3.329053,91,0.618475,31,None,pmgr,0.505185


In [251]:
f1_list

[0.6318518518518519,
 0.5455555555555556,
 0.6055555555555555,
 0.562962962962963,
 0.695925925925926,
 0.5711111111111111,
 0.5248148148148148,
 0.6196296296296296,
 0.5051851851851852,
 0.7725925925925926]

In [260]:
theta_list

[0.7824170924598155,
 0.8340306113890947,
 0.9585674041259294,
 0.6293604347180899,
 0.9123723785617636,
 0.8663436685722214,
 0.9362941960129322,
 0.7788576835572925,
 0.6184749109756382,
 0.9543071678016238]

In [223]:
statistics.mean(f1_list)

0.6035185185185186

In [ ]:
def computeWrongProb(wrongProbTuple):
    sumProbDiff = []
    for item in wrongProbTuple:
        predictID = np.argmax(item[1])
        predictProb = item[0][0][predictID]
        trueID = np.argmax(item[2])
        trueProb = item[0][0][trueID]
        sumProbDiff.append(predictProb - trueProb)

    #print(statistics.mean(sumProbDiff))
    print(len(sumProbDiff))
    
    return 0

    
    
def probDiffWrong(rows):
    length = rows.shape[0]

    wrongProbDiff = []
        
    for index, row in rows.iterrows():
        
        prob = row["Prob"].split("/")
        trueID = row["Real_Goal"]
        results = row["Results"].split("/")
        all_candidates = row["Cost"].split("/")
        
        total = len(all_candidates)-1   # the last one is /
        
        bacc = func_bacc(total, results, trueID)
        r = func_recall(results, trueID)
        
        if bacc != 1:
            
            counter = 0
            prob_sum = 0
            maxProb = 0
            for i in results:
                if i:
                    if float(prob[int(i)]) > maxProb:
                        maxProb = float(prob[int(i)])
                    counter += 1
                    prob_sum += float(prob[int(i)])
            averageProb = prob_sum/counter
            trueProb = float(prob[trueID])
            # print(averageProb)
            # wrongProbDiff.append(averageProb-trueProb)
            wrongProbDiff.append(maxProb-trueProb)
    
    return wrongProbDiff

        
        
# when it's wrong
def calculate_wrong_cases(rows):
    length = rows.shape[0]
    
    wrongProbDiff = []
    for index, row in rows.iterrows():
        
        prob = row["Prob"].split("/")
        answer = row["Real_Goal"]
        results = row["Results"].split("/")
        all_candidates = row["Cost"].split("/")
        total = len(all_candidates)-1   # the last one is /
        
        p = func_precision(results, answer)
        r = func_recall(results, answer)
        f = func_f1(total, results, answer)
        a = func_accuracy(total, results, answer)
        bacc = func_bacc(total, results, answer)
        
        if bacc != 1:
            maxProb = 0
            for i in results:
                if i and float(prob[int(i)]) > maxProb:
                    maxProb = float(prob[int(i)])
                    
            trueProb = float(prob[int(answer)])
            wrongProbDiff.append(maxProb-trueProb)
    
    return wrongProbDiff

# Remove the last 10 points from source data

In [44]:
## remove last 10 datapoints

# sort the df to traces for every subject and goals
def extract_traces(dfn):
    traces = []

    Subject = 0
    Loc = 0
    Iteration = 0
    tup = (Subject, Loc, Iteration)

    for index, row in dfn.iterrows():
        curr_Subject = row["Subject"]
        curr_Loc = row["Loc"]
        curr_Iteration = row["Iteration"]
        curr_tup = (curr_Subject, curr_Loc, curr_Iteration)

        if curr_tup != tup:
            #print("new trace")
            tup = curr_tup

            rslt_df = dfn[(dfn['Subject'] == curr_Subject) 
                      & (dfn['Loc'] == curr_Loc) 
                      & (dfn['Iteration'] == curr_Iteration)]

            rslt_df.reset_index(drop=True, inplace=True)
            traces.append(rslt_df)
            
    return traces


input_data_file = "Corrected_UpperLimbReachingData_AddVelocity&FinalStatic.csv"
output_data_file = "corrected_final_input_data.csv"

df = pd.read_csv(input_data_file)
df_traces = extract_traces(df)

df_remove_10_last = pd.DataFrame([])

for df_a_trace in df_traces:
    df_remove_10_last = pd.concat([df_remove_10_last, df_a_trace.iloc[0:-10,:]], axis=0)
    
df_remove_10_last.reset_index(drop=True, inplace=True)

df_remove_10_last.to_csv(output_data_file, index=False)

# Confidence interval

In [65]:
import scipy.stats as st

def confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = sum(data) / n
    std_error = st.sem(data)
    margin_of_error = std_error * st.t.ppf((1 + confidence) / 2, n - 1)
    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error
    return lower_bound, upper_bound, margin_of_error

# Example usage
data = [2, 4, 6, 8, 10]
lower, upper, margin_of_error= confidence_interval(data)

print(f"Confidence Interval: [{lower}, {upper}]")

Confidence Interval: [2.073513677044878, 9.926486322955121]


In [66]:
data = 
_, _, margin_of_error= confidence_interval(data)
margin_of_error

3.926486322955122